In [310]:
#implementation of Modern Portfolio Theory
import scipy.optimize as sc
import numpy as np
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import plotly.graph_objects as go

In [415]:
def getStats(stocks):
    returns = stocks.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    
    return meanReturns, covMatrix

In [357]:
def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    #요건 공식을 바야 할 듯
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights))) * np.sqrt(252)
    return returns, std

In [358]:
#음수 샤프 비율
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0.):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return -(pReturns - riskFreeRate)/pStd

def maxSR(meanReturns, covMatrix, riskFreeRate = 0., constraintSet=(0,1)):
    "Minimize the negative SR, by altering the weights of the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x)-1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.minimize(negativeSR, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [359]:
def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, riskFreeRate = 0, constraintSet=(0,1)):
    "Minimize the portfolio variance by altering the weights/allocation of assets in the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x)-1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.minimize(portfolioVariance, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def portfolioReturn(weights):
        return portfolioPerformance(weights, meanReturns, covMatrix)[0]

In [314]:
stockList = ['CBA', 'BHP', 'TLS']
stocks = [stock+'.AX' for stock in stockList]

In [315]:
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

In [360]:
df=pd.read_excel('2000_2021_close.xlsx')

In [317]:
df.drop(['Code'],axis=1, inplace=True)

In [412]:
df2 = df.iloc[:,:10]
df2

,A000020,A000040,A000050,A000060,A000070,A000080,A000100,A000120,A000140,A000150
0,3116,26978,1694,650,13177,4515.0,3223,34118,37917,26372
1,3305,30867,1651,748,13177,4190.0,3164,34808,37917,26668
2,2871,33055,1723,724,13069,4200.0,2956,32875,36315,25088
3,3296,29652,1644,728,13717,4000.0,3095,33703,35033,25631
4,3211,32082,1622,836,14689,4300.0,3118,33151,36101,25433
...,...,...,...,...,...,...,...,...,...,...
5365,18050,1050,13650,29250,116500,33300.0,62800,163000,14300,98100
5366,16500,1025,13550,30000,113500,33350.0,61200,159500,14200,102000
5367,18550,1015,13550,29700,112000,33000.0,60900,156500,14100,98000
5368,17850,1015,13800,30550,111500,33150.0,61500,153000,14150,99600


In [416]:
meanReturns, covMatrix = getStats(df2)

In [417]:
print(meanReturns)

A000020    0.000769
A000040    0.000273
A000050    0.000767
A000060    0.001125
A000070    0.000751
A000080    0.005183
A000100    0.000782
A000120    0.000747
A000140    0.000222
A000150    0.000728
dtype: float64


In [419]:
lenT = len(df2.columns)
weights = lenT*[1./lenT] 
print(weights)

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]


In [352]:
def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)):
    """for each returnTarget, we want to optimise the portfolio for min variance"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    
    constraints = ({'type':'ineq', 'fun':lambda x: portfolioReturn(x) - returnTarget},
                   {'type': 'eq', 'fun': lambda x: np.sum(x)-1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    effOpt = sc.minimize(portfolioVariance,numAssets*[1./numAssets], args=args,
                        method = 'SLSQP', bounds=bounds, constraints=constraints)
    
    return effOpt

In [322]:
def calculatedResults(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Read in mean, cov matrix, and other financial information
        Output, Max SR , Min Volatility, efficient frontier """
    # Max Sharpe Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation]
    
    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]

    # Efficient Frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun'])

    maxSR_returns, maxSR_std = round(maxSR_returns*100,2), round(maxSR_std*100,2)
    minVol_returns, minVol_std = round(minVol_returns*100,2), round(minVol_std*100,2)

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns

In [420]:
print(calculateResults(meanReturns, covMatrix))

(22.74, 25.19,          allocation
A000020         6.0
A000040         0.0
A000050        17.0
A000060        26.0
A000070        10.0
A000080         1.0
A000100        30.0
A000120         8.0
A000140         0.0
A000150         2.0, 18.04, 23.05,          allocation
A000020         6.0
A000040         4.0
A000050        16.0
A000060        10.0
A000070        10.0
A000080         0.0
A000100        28.0
A000120         9.0
A000140        13.0
A000150         4.0, [0.23050045123944096, 0.2305584409637852, 0.23073125847096046, 0.23101822150314966, 0.23141924691641172, 0.23193391312465175, 0.2325605987292041, 0.23329919704524138, 0.23414844415623168, 0.2351070992051853, 0.23617383212904855, 0.23734718960521897, 0.23862560542299258, 0.24000741293061087, 0.24149101460814854, 0.243085823669085, 0.2448557094212482, 0.24691011947645408, 0.24924467207696818, 0.25185158015773035], array([0.18044618, 0.18291877, 0.18539135, 0.18786394, 0.19033653,
       0.19280912, 0.19528171, 0.1977543 , 0.2

In [ ]:
def EF_graph(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Return a graph ploting the min vol, max sr and efficient frontier"""
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns = calculatedResults(meanReturns, covMatrix, riskFreeRate, constraintSet)

    #Max SR
    MaxSharpeRatio = go.Scatter(
        name='Maximium Sharpe Ratio',
        mode='markers',
        x=[maxSR_std],
        y=[maxSR_returns],
        marker=dict(color='red',size=14,line=dict(width=3, color='black'))
    )

    
    
    
    
    #Min Vol
    MinVol = go.Scatter(
        name='Mininium Volatility',
        mode='markers',
        x=[minVol_std],
        y=[minVol_returns],
        marker=dict(color='green',size=14,line=dict(width=3, color='black'))
    )

    #Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(ef_std*100, 2) for ef_std in efficientList],
        y=[round(target*100, 2) for target in targetReturns],
        line=dict(color='black', width=4, dash='dashdot')
    )

    data = [MaxSharpeRatio, MinVol, EF_curve]

    layout = go.Layout(
        title = 'Portfolio Optimisation with the Efficient Frontier',
        yaxis = dict(title='Annualised Return (%)'),
        xaxis = dict(title='Annualised Volatility (%)'),
        showlegend = True,
        legend = dict(
            x = 0.75, y = 0, traceorder='normal',
            bgcolor='#E2E2E2',
            bordercolor='black',
            borderwidth=2),
        width=800,
        height=600)
    
    fig = go.Figure(data=data, layout=layout)
    return fig.show()

In [421]:
EF_graph(meanReturns, covMatrix)

In [60]:
#최적화 목적함수
def obj(x):
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    x4 = x[3]
    return x1*x4*(x1+x2+x3)+x3

#0을 기준으로 제약조건 작성 0보다 크거나 같게 ㄱㄱ
def const1(x):
    return x[0]*x[1]*x[2]*x[3]-25

#x끼리 제곱한거 더한 합이 40이어야 함
def const2(x):
    sum_sq = np.sum(np.square(x))
    return sum_sq-40

#초기값은 리스트 형태로
x0 = [1, 5, 5, 1]

#해의 범위는 튜플로 만들어 정함. 변수가 하나니까 하나의 범위만 지정하면 됨
b = (1, 5)
#한개 변수 범위를 반복해서 지정해주어야 함
bnd = (b, b, b, b)

#제약조건은 딕셔너리 형태로. type은 eq , 제약식 함수값 = 0 ineq 는 제약식 함수값이 0이 아닐때
#fun: 제약식 함수를 값으로 입력
#그리고 딕셔너리 형태를 리스트로 묶어줌. 
con1  = {'type':'ineq', 'fun':const1}
con2  = {'type':'eq', 'fun':const2}
cons = [con1, con2]

sol=sc.minimize(obj, x0, method='SLSQP', bounds=bnd, constraints=cons)

최적화란 특정 집합 위에서 정의된 실숫값, 함수, 정수에 대해 그 값이 최대나 최소가 되는 상태를 해석하는 문제. 즉, 최적의 값을 찾아내는 것

최적화를 위한 첫번째는 문제 정의(목적함수, 제약조건)
목적함수는 최소화, 최대화가 되는 대상. 제약조건은 결과가 도출되어야 하는 범위 및 조건

SLSQP(Sequential Least SQaures Programming)은 복잡한 현실 문제를 단순화해 목적함수를 이차식으로 근사해 풀고 다음번 지점을 예측해 동일한 방법을 수행하는 문제를 푸는 알고리즘. 비선형 최적화 문제를 해결할 수 있다. 제약조건(constraint)와 상하한선(bound)가 있는 조건에서 사용할 수 있는 알고리즘이다. 

